# Модель обнаружения отклонений в ИП

In [ ]:
from __future__ import division

Подключаем основные библиотеки для работы с данными

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import seaborn as sns

## Загрузка данных
Показания сенсоров храняться в одном csv файле. В качестве индекса используется столбце времени *time*

In [ ]:
train = pd.read_csv('/media/volume/datasets/UpAndDown/train.csv', index_col='time')

## Описание

Точек по времени

In [ ]:
N = len(train)
N

Размерность временного ряда

In [ ]:
K = len(train.columns)
K

Размер занимаемой RAM (Гб)

In [ ]:
print("{:.2f}".format(train.memory_usage(index=True).sum()/2**30))

## Визуализация данных
Для визуализации выберем из всего набора несколько основных рядов

In [ ]:
columns_to_plot = [
    'tank.level',
    'valveDiscrete1.open',
    'valveDiscrete1.V_flow',
    'valveDiscrete2.open',
    'valveDiscrete2.V_flow',
]

Построим графики для выбранных временных рядов на первых 

In [ ]:
end_plot = 10000

точках

In [ ]:
_ = train[columns_to_plot][:end_plot].plot(subplots=True, figsize=(15,10), color='blue')


### Предобработка данных

Поскольку для построения прогноза будут использоваться нейронные сети, то необходимо нормализовать все временные ряды. Для этого будет использовать инструменты библиотеки sklearn

In [ ]:
from sklearn.preprocessing import StandardScaler
standart_scaler = StandardScaler()
train_scaled = pd.DataFrame(data=standart_scaler.fit_transform(train), index=train.index, columns=train.columns)

### Визуализируем результат нормализации

In [ ]:
_ = train_scaled[columns_to_plot][:end_plot].plot(subplots=True, figsize=(15,10), color='blue')

### Разделение данных на обучающую и проверочную выборки

Пусть доля проверочной выборки составляет

In [ ]:
validation_split = 0.1

Число точек в обучающей выборке

In [ ]:
train_size = int(math.floor(N*(1 - validation_split)))
train_size

Число точек в проверочной выборке

In [ ]:
validation_size = int(math.ceil(N*validation_split))
validation_size

In [ ]:
assert(validation_size + train_size == N)

In [ ]:
train_dataset = train_scaled.iloc[:train_size]
validation_dataset = train_scaled.iloc[train_size:]

Будем обрабатывать данные окнами по

In [ ]:
window_size = 60

точек.

In [ ]:
def crop(dataset, window_size):
    crop_size = len(dataset) - len(dataset) % window_size
    return dataset.iloc[:crop_size].values.reshape(-1, window_size, K)

In [ ]:
train_reshaped = crop(train_dataset, window_size)
X_train = train_reshaped[:-1]
y_train = train_reshaped[1:]
assert(X_train.shape == y_train.shape)
validation_reshaped = crop(validation_dataset, window_size)
X_validation = validation_reshaped[:-1]
y_validation = validation_reshaped[1:]
assert(X_validation.shape == y_validation.shape)

## Обучение RNN
Решение описанной задачи будем искать с помощью построения прогноза для временого ряда и оценки полученной ошибки между реальным значением и спрогнозированным.

### Архитектура нейронной сети

Для построения необходимой архитектуры будем использовать библиотеку keras

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.layers.wrappers import TimeDistributed

Сеть будет иметь два скрытых LSTM слоя размерностью

In [ ]:
n_hidden = (32, 32)

Для регуляризации параметров будем использовать метод Dropout для каждого скрытого слоя с вероятностью

In [ ]:
dropout_prob = 0.5

Обучение и вывод будем проводить при размере батча в

In [ ]:
batch_size = 1

Размерность входных данных

In [ ]:
input_shape = (batch_size, window_size, K)

In [ ]:
model = Sequential()
model.add(LSTM(batch_input_shape=input_shape,
               output_dim=n_hidden[0],
               stateful=True,
               return_sequences=True))
model.add(Dropout(dropout_prob))
model.add(LSTM(output_dim=n_hidden[1],
               stateful=True,
               return_sequences=True))
model.add(TimeDistributed(Dense(K)))
model.add(Activation('linear'))

В качестве функции потерь будем использовать средний квадрат ошибка. В качестве алгоритма потимизации RmsProp

In [ ]:
model.compile(loss='mse', optimizer='RMSprop')

### Параметры нейронной сети

In [ ]:
model.summary()

Максимальное число эпох обучения

In [ ]:
max_epoch = 100

В ходе обучения будем сохранять лучший результат на основе валидации. И будем преждевременно заканчивать обучение, если в ходе нескольких эпох нет улучшения результата.

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpointer = ModelCheckpoint(filepath="./models/lstm.multi.UpAndDown.{:d}-{:d}.hdf5".format(n_hidden[0], n_hidden[1]),
                               verbose=0,
                               save_best_only=True)
early_stopper = EarlyStopping(patience=10, verbose=1)

In [ ]:
history = model.fit(X_train,
                   y_train,
                   batch_size=batch_size,
                   nb_epoch=max_epoch,
                   validation_data=(X_validation, y_validation),
                   shuffle=False,
                   verbose=1,
                   callbacks=[checkpointer, early_stopper])

Построим график зависимости функции потерь на обучающей и проверочной выборках от номера эпохи

In [ ]:
pd.DataFrame(history.history).plot(figsize=(10, 3))

Для проверочной выборки построим прогноз

In [ ]:
model.load_weights("./models/lstm.multi.UpAndDown.{:d}-{:d}.hdf5".format(n_hidden[0], n_hidden[1]))
model.reset_states()
y_pred = model.predict(X_validation, batch_size=batch_size)
pred_scaled = pd.DataFrame(index=validation_dataset.index[window_size:window_size + y_pred.shape[0]*y_pred.shape[1]])
for i in range(K):
    pred_scaled[train.columns[i]] = y_pred[:, :, i].reshape(-1)

Визуализируем прогноз на валидационной выборке и построим график средней ошибки прогноза по всем измерениям

In [ ]:
def plot_forecast_with_error(true, pred, error, threshold):
    f, ax = plt.subplots(len(columns_to_plot) + 1, figsize=(15, 2*len(columns_to_plot)))
    
    for i, column in enumerate(true.columns):
        true[column].plot(ax=ax[i], color='blue', label=None)
        pred[column].plot(ax=ax[i], color='green', label=None)
    error.plot(ax=ax[-1], color='red', label='mean_error')
    pd.Series(data=[threshold]*len(true.index), index=true.index).plot(ax=ax[-1], color='orange', label='threshold')

In [ ]:
start_plot = 490000
end_plot = 500000
error = ((validation_dataset.iloc[window_size:window_size + y_pred.shape[0]*y_pred.shape[1]] - pred_scaled)**2).ewm(halflife=window_size).mean()
mean_error = error.mean(axis=1)

plot_forecast_with_error(validation_dataset[columns_to_plot][start_plot:end_plot],
                         pred_scaled[columns_to_plot][start_plot:end_plot],
                         mean_error[start_plot:end_plot],
                         threshold)

Определим порог принятия решения как 0.999 квантиль от ошибки на валидации

In [ ]:
threshold = mean_error.quantile(0.999)
print("{:.3g}".format(threshold))

умноженный на

In [ ]:
alpha = 2.0
threshold_level = alpha*threshold

### Тестирование алгоритма обнаружения атаки

Загрузим данные, содержащие последствия атаки и сделаем предобработку

In [ ]:
test = pd.read_csv('/media/volume/datasets/UpAndDown/1/test6.csv', index_col='time')
test_scaled = pd.DataFrame(data=standart_scaler.transform(test), index=test.index, columns=test.columns)
test_reshaped = crop(test_scaled, window_size)
X_test = test_reshaped[:-1]

Построим прогноз для проверочного набора

In [ ]:
model.reset_states()
test_pred = model.predict(X_test, batch_size=batch_size)
test_pred_scaled = pd.DataFrame(index=test.index[window_size:window_size + test_pred.shape[0]*test_pred.shape[1]])
for i in range(K):
    test_pred_scaled[train.columns[i]] = test_pred[:, :, i].reshape(-1)

И визуализируем результат

In [ ]:
start_plot = 3000
end_plot = -1
error = ((test_scaled.iloc[window_size:window_size + test_pred_scaled.shape[0]*test_pred_scaled.shape[1]] - test_pred_scaled)**2).ewm(halflife=window_size).mean()
test_mean_error = error.mean(axis=1)

plot_forecast_with_error(test_scaled[columns_to_plot][start_plot:],
                         test_pred_scaled[columns_to_plot][start_plot:],
                         test_mean_error[start_plot:],
                         threshold_level)